<a href="https://colab.research.google.com/github/ronaknavadiya/pytorch-projects-practice-/blob/main/pytorch_RNN_based_qa_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [15]:
# tokenize

def tokenize(text):
  text = text.lower()
  text = text.replace("'", '')
  text = text.replace('?', '')
  return text.split()

In [16]:
# vocab

vocab = {'<UNK>': 0}

def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:
    if token not in vocab:
      vocab[token] = len(vocab)


In [17]:
df.apply(build_vocab, axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [19]:
len(vocab)

324

In [24]:
# convert text to numbers array

def text_to_indices(text, vocab):
  indexed_text = []
  for token in tokenize(text):
    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])
  return indexed_text

In [25]:
import torch
from torch.utils.data import Dataset, DataLoader

In [26]:
class QADataset(Dataset):
  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    question = text_to_indices(self.df.iloc[idx]["question"], self.vocab)
    answer = text_to_indices(self.df.iloc[idx]["answer"], self.vocab)
    return torch.tensor(question), torch.tensor(answer)



In [70]:
dataset = QADataset(df, vocab)

In [27]:
dataset[0]

(tensor([1, 2, 3, 4, 5, 6]), tensor([7]))

In [28]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [74]:
for question, answer in dataloader:
  print(question)
  print(answer[0])
  print(answer[0].shape)
  break

tensor([[  1,   2,   3, 122, 123,  19,   3,  45]])
tensor([124])
torch.Size([1])


In [29]:
import torch.nn as nn

In [81]:
class SimpleRNN(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)  # fully connected layer

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, output = self.rnn(embedded_question)
    final_output = self.fc(output.squeeze(0))
    return final_output

In [82]:
LR = 0.01
epochs = 20

In [83]:
model = SimpleRNN(len(vocab))
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [85]:
# training loop

for epoch in range(epochs):
  total_loss = 0
  for question, answer in dataloader:
    optimizer.zero_grad()

    # forward pass
    pred = model(question)

    loss = loss_fn(pred, answer[0])

    # backward pass
    loss.backward()

    # upgrade gradient
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss/len(dataloader)}")

Epoch: 1, Loss: 0.011020300958383208
Epoch: 2, Loss: 0.005607859232178372
Epoch: 3, Loss: 0.0042593101608670424
Epoch: 4, Loss: 0.003657052674356641
Epoch: 5, Loss: 0.003228904054655383
Epoch: 6, Loss: 0.0029026453072826066
Epoch: 7, Loss: 0.0026189676104372162
Epoch: 8, Loss: 0.002405798714607954
Epoch: 9, Loss: 0.0022144175710208302
Epoch: 10, Loss: 0.0020395567246143604
Epoch: 11, Loss: 0.00190275197253666
Epoch: 12, Loss: 0.0017769782972108159
Epoch: 13, Loss: 0.0016628939199209627
Epoch: 14, Loss: 0.001561236078850925
Epoch: 15, Loss: 0.0014579423845538662
Epoch: 16, Loss: 0.001379218793590553
Epoch: 17, Loss: 0.0012923885356738336
Epoch: 18, Loss: 0.001221212697823325
Epoch: 19, Loss: 0.0011537780115355013
Epoch: 20, Loss: 0.001091142181151857


In [106]:
def predict(question, model, threshold = 0.5):
  question_numbers = text_to_indices(question, vocab)
  question = torch.tensor(question_numbers).unsqueeze(0)

  pred = model(question)
  # print(pred) # it will give logits

  # convert logits to probability
  prob = nn.functional.softmax(pred, dim=1)

  #  find index of max prob
  max_prob, max_idx = torch.max(prob, dim=1)

  if max_prob > threshold:
    return list(vocab.keys())[max_idx]
  else:
    return "I don't know"


In [109]:
predict("largest planet in solar system", model)

'jupiter'